In [14]:
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix



## load tested data


In [15]:
# Load the data from the .csv file
data = pd.read_csv('cleaned_descriptor_data',index_col=0)
X_fingerprints=pd.read_csv('fingerprint_data',index_col=0)

# Scale the data with a standard scaling
scaling=MinMaxScaler()
scaling.fit(data)
scaled_data=scaling.transform(data)
df_data = pd.DataFrame(scaled_data, columns=data.columns,index=data.index)

# Choose the kinase of which you want to predict the inhibition
kinase_ERK2 = 'ERK2_inhibition'
kinase_PKM2 = 'PKM2_inhibition'    

# Define train and test data
X = df_data.drop(columns=['ERK2_inhibition','PKM2_inhibition']).copy()
y_pkm2 = data[kinase_PKM2].copy()
y_erk2 = data[kinase_ERK2].copy()


## Load untested data

In [16]:
untested_molecules=pd.read_csv('cleaned_descriptor_data_untested',index_col=0)
untested_molecules.head()
X_untested=untested_molecules.drop(columns=['ERK2_inhibition','PKM2_inhibition']).copy()

y_untested_erk2=untested_molecules[kinase_ERK2].copy()
y_untested_pkm2=untested_molecules[kinase_PKM2].copy()

In [17]:
X_untested_fingerprints=pd.read_csv('fingerprint_data_untested',index_col=0)

## Recursive Feature Exclusion

In [18]:
rfe_pmk = RFE(RandomForestClassifier(), n_features_to_select=50)
rfe_pmk.fit(X, y_pkm2)
selected_features_pmk = X.columns[rfe_pmk.support_]
print(selected_features_pmk)

rfe_erk = RFE(RandomForestClassifier(), n_features_to_select=50)
rfe_erk.fit(X, y_erk2)
selected_features_erk = X.columns[rfe_erk.support_]
print(selected_features_erk)

Index(['MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS',
       'MinPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2',
       'FpDensityMorgan3', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
       'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW',
       'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0n', 'Chi1v', 'Chi3n', 'Chi3v',
       'HallKierAlpha', 'Ipc', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA3',
       'PEOE_VSA5', 'PEOE_VSA7', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10',
       'SMR_VSA3', 'SMR_VSA7', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA2',
       'SlogP_VSA5', 'TPSA', 'EState_VSA3', 'EState_VSA6', 'VSA_EState1',
       'VSA_EState4', 'VSA_EState5', 'VSA_EState6', 'VSA_EState9',
       'NumAromaticHeterocycles', 'MolLogP', 'fr_Ar_N'],
      dtype='object')
Index(['MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed',
       'HeavyAtomMolWt', 'MinPartialCharge', 'MinAbsPartialCharge',
       'FpDensityMorgan1', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 

In [19]:
X_cleaned_pmk=X[selected_features_pmk]
X_untested_cleaned_pmk=X_untested[selected_features_pmk]

X_cleaned_erk=X[selected_features_erk]
X_untested_cleaned_erk=X_untested[selected_features_erk]

# Random forests

## Random forest with descriptors evaluation

### Find n_estimators

In [20]:
# ERK2
X_train, X_test, y_train, y_test = train_test_split(X_cleaned_erk, y_erk2, test_size=0.2, stratify=y_erk2, random_state=40)

pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=10)],['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=40)
    
param_grid = {'classifier__n_estimators':[100, 200, 300, 400, 500, 600]}
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='recall', cv=stratified_kfold, n_jobs=-1)

grid_search.fit(X_train, y_train)
n_estimators_erk = grid_search.best_params_['classifier__n_estimators']
print(n_estimators_erk)

# PMK2
X_train_pmk, X_test_pmk, y_train_pmk, y_test_pmk = train_test_split(X_cleaned_pmk, y_pkm2, test_size=0.2, stratify=y_pkm2, random_state=40)

pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=10)],['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=40)
    
param_grid = {'classifier__n_estimators':[100, 200, 300, 400, 500, 600]}
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='recall', cv=stratified_kfold, n_jobs=-1)

grid_search.fit(X_train_pmk, y_train_pmk)
n_estimators_pmk = grid_search.best_params_['classifier__n_estimators']
print(n_estimators_pmk)


200
100


In [21]:
#ERK2:
# Split the data into training and test sets
X_train_erk, X_test_erk, y_train_erk, y_test_erk = train_test_split(X_cleaned_erk, y_erk2, test_size=0.2)

# Apply SMOTE to balance the training set
smote_erk = SMOTE(random_state=40)
X_train_balanced_erk, y_train_balanced_erk = smote_erk.fit_resample(X_train_erk, y_train_erk)

# Calculate the ratio between 0 and 1 in the balanced training set and the test set
ratio_train_balanced_erk = y_train_balanced_erk.value_counts(normalize=True)
ratio_test_erk = y_test_erk.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the balanced training set:")
print(ratio_train_balanced_erk)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test_erk)

# Train the Random Forest model with the balanced training set
rf_descriptors = RandomForestClassifier(n_estimators_erk)

# Perform cross-validation
cv = StratifiedKFold(n_splits=2, shuffle=False)
cv_scores = cross_val_score(rf_descriptors, X_train_balanced_erk, y_train_balanced_erk, cv=cv, scoring='recall')

print(f'Cross-Validation Scores: {cv_scores}')

rf_descriptors.fit(X_train_balanced_erk, y_train_balanced_erk)
y_pred_erk = rf_descriptors.predict(X_test_erk)

# Evaluate the model using a confusion matrix
conf_mat_erk = confusion_matrix(y_test_erk, y_pred_erk)
print("Confusion Matrix:")
print(conf_mat_erk)
print(classification_report(y_test_erk, y_pred_erk))


#PMK:
# Split the data into training and test sets
X_train_pmk, X_test_pmk, y_train_pmk, y_test_pmk = train_test_split(X_cleaned_pmk, y_pkm2, test_size=0.2)

# Apply SMOTE to balance the training set
smote_pmk = SMOTE(random_state=40)
X_train_balanced_pmk, y_train_balanced_pmk = smote_pmk.fit_resample(X_train_pmk, y_train_pmk)

# Calculate the ratio between 0 and 1 in the balanced training set and the test set
ratio_train_balanced_pmk = y_train_balanced_pmk.value_counts(normalize=True)
ratio_test_pmk = y_test_pmk.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the balanced training set:")
print(ratio_train_balanced_pmk)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test_pmk)

# Train the Random Forest model with the balanced training set
rf_descriptors_pmk = RandomForestClassifier(n_estimators_pmk, random_state=42)

# Perform cross-validation
cv = StratifiedKFold(n_splits=2, shuffle=False)
cv_scores_pmk= cross_val_score(rf_descriptors, X_train_balanced_pmk, y_train_balanced_pmk, cv=cv, scoring='recall')

print(f'Cross-Validation Scores: {cv_scores_pmk}')

rf_descriptors_pmk.fit(X_train_balanced_pmk, y_train_balanced_pmk)
y_pred_pmk = rf_descriptors_pmk.predict(X_test_pmk)

# Evaluate the model using a confusion matrix
conf_mat_pmk = confusion_matrix(y_test_pmk, y_pred_pmk)
print("Confusion Matrix:")
print(conf_mat_pmk)
print(classification_report(y_test_pmk, y_pred_pmk))

Ratio of 0 and 1 inhibitors in the balanced training set:
ERK2_inhibition
0    0.5
1    0.5
Name: proportion, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
ERK2_inhibition
0    0.96875
1    0.03125
Name: proportion, dtype: float64
Cross-Validation Scores: [0.98584906 0.98820755]
Confusion Matrix:
[[216   1]
 [  7   0]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       217
           1       0.00      0.00      0.00         7

    accuracy                           0.96       224
   macro avg       0.48      0.50      0.49       224
weighted avg       0.94      0.96      0.95       224

Ratio of 0 and 1 inhibitors in the balanced training set:
PKM2_inhibition
0    0.5
1    0.5
Name: proportion, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
PKM2_inhibition
0    0.982143
1    0.017857
Name: proportion, dtype: float64
Cross-Validation Scores: [0.99308756 1.        ]
Confusion Matrix:
[[218   2]
 [  3

## Random forest with fingerprints evaluation

In [22]:
#ERK2:
# Split the data into training and test sets
X_train_erk, X_test_erk, y_train_erk, y_test_erk = train_test_split(X_fingerprints, y_erk2, test_size=0.2)

# Apply SMOTE to balance the training set
smote_erk = SMOTE(random_state=40)
X_train_balanced_finger_erk, y_train_balanced_finger_erk = smote_erk.fit_resample(X_train_erk, y_train_erk)

# Calculate the ratio between 0 and 1 in the balanced training set and the test set
ratio_train_balanced_erk = y_train_balanced_erk.value_counts(normalize=True)
ratio_test_erk = y_test_erk.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the balanced training set:")
print(ratio_train_balanced_erk)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test_erk)

# Train the Random Forest model with the balanced training set
rf_finger_erk = RandomForestClassifier(n_estimators_erk, random_state=42)

# Perform cross-validation
cv = StratifiedKFold(n_splits=2, shuffle=False)
cv_scores = cross_val_score(rf_descriptors, X_train_balanced_finger_erk, y_train_balanced_finger_erk, cv=cv, scoring='recall')

print(f'Cross-Validation Scores: {cv_scores}')

rf_finger_erk.fit(X_train_balanced_finger_erk, y_train_balanced_finger_erk)
y_pred_erk = rf_finger_erk.predict(X_test_erk)

# Evaluate the model using a confusion matrix
conf_mat_erk = confusion_matrix(y_test_erk, y_pred_erk)
print("Confusion Matrix:")
print(conf_mat_erk)
print(classification_report(y_test_erk, y_pred_erk))


#PMK:
# Split the data into training and test sets
X_train_pmk, X_test_pmk, y_train_pmk, y_test_pmk = train_test_split(X_fingerprints, y_pkm2, test_size=0.2)

# Apply SMOTE to balance the training set
smote_pmk = SMOTE(random_state=40)
X_train_balanced_finger_pmk, y_train_balanced_finger_pmk = smote_pmk.fit_resample(X_train_pmk, y_train_pmk)

# Calculate the ratio between 0 and 1 in the balanced training set and the test set
ratio_train_balanced_pmk = y_train_balanced_pmk.value_counts(normalize=True)
ratio_test_pmk = y_test_pmk.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the balanced training set:")
print(ratio_train_balanced_pmk)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test_pmk)

# Train the Random Forest model with the balanced training set
rf_finger_pmk = RandomForestClassifier(n_estimators_pmk, random_state=42)

# Perform cross-validation
cv = StratifiedKFold(n_splits=2, shuffle=False)
cv_scores_pmk= cross_val_score(rf_descriptors, X_train_balanced_finger_pmk, y_train_balanced_finger_pmk, cv=cv, scoring='recall')

print(f'Cross-Validation Scores: {cv_scores_pmk}')

rf_finger_pmk.fit(X_train_balanced_finger_pmk, y_train_balanced_finger_pmk)
y_pred_pmk = rf_finger_pmk.predict(X_test_pmk)

# Evaluate the model using a confusion matrix
conf_mat_pmk = confusion_matrix(y_test_pmk, y_pred_pmk)
print("Confusion Matrix:")
print(conf_mat_pmk)
print(classification_report(y_test_pmk, y_pred_pmk))

Ratio of 0 and 1 inhibitors in the balanced training set:
ERK2_inhibition
0    0.5
1    0.5
Name: proportion, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
ERK2_inhibition
0    0.950893
1    0.049107
Name: proportion, dtype: float64
Cross-Validation Scores: [0.94600939 1.        ]
Confusion Matrix:
[[207   6]
 [ 10   1]]
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       213
           1       0.14      0.09      0.11        11

    accuracy                           0.93       224
   macro avg       0.55      0.53      0.54       224
weighted avg       0.91      0.93      0.92       224

Ratio of 0 and 1 inhibitors in the balanced training set:
PKM2_inhibition
0    0.5
1    0.5
Name: proportion, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
PKM2_inhibition
0    0.973214
1    0.026786
Name: proportion, dtype: float64
Cross-Validation Scores: [0.97701149 1.        ]
Confusion Matrix:
[[218   0]
 [ 

c:\Users\20224085\.conda\envs\groupass_group8\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\20224085\.conda\envs\groupass_group8\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\20224085\.conda\envs\groupass_group8\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

## Random forest with descriptors on actual data

In [23]:
smote_pmk = SMOTE(random_state=40)
X_balanced_pmk, y_balanced_pmk = smote_pmk.fit_resample(X_cleaned_pmk, y_pkm2)

rf_descriptors_pmk = RandomForestClassifier(n_estimators_pmk)
rf_descriptors_pmk.fit(X_balanced_pmk, y_balanced_pmk)
y_pred_descriptor_pmk = rf_descriptors_pmk.predict(X_untested_cleaned_pmk)
df_y_pred_descriptor_pmk=pd.DataFrame(y_pred_descriptor_pmk,index=untested_molecules.index)

smote_erk = SMOTE(random_state=40)
X_balanced_erk, y_balanced_erk = smote_erk.fit_resample(X_cleaned_erk, y_erk2)

rf_descriptors_erk = RandomForestClassifier(n_estimators_erk)
rf_descriptors_erk.fit(X_balanced_erk, y_balanced_erk)
y_pred_descriptor_erk = rf_descriptors.predict(X_untested_cleaned_erk)
df_y_pred_descriptor_erk=pd.DataFrame(y_pred_descriptor_erk,index=untested_molecules.index)


## Random forest with fingerprints on actual data

In [24]:
smote_erk = SMOTE(random_state=40)
X_fingerprints_pkm, y_fingerprints_pkm = smote_erk.fit_resample(X_fingerprints, y_pkm2)
rf_fingerprints_pmk = RandomForestClassifier(n_estimators_pmk)
rf_fingerprints_pmk.fit(X_fingerprints_pkm, y_balanced_pmk)
y_pred_fingerprint_pmk = rf_fingerprints_pmk.predict(X_untested_fingerprints)
df_y_pred_fingerprint_pmk=pd.DataFrame(y_pred_fingerprint_pmk,index=untested_molecules.index)

smote_erk = SMOTE(random_state=40)
X_fingerprints_erk, y_fingerprints_erk = smote_erk.fit_resample(X_fingerprints, y_erk2)
rf_fingerprints_erk = RandomForestClassifier(n_estimators_erk)
rf_fingerprints_erk.fit(X_fingerprints_erk, y_balanced_erk)
y_pred_fingerprint_erk = rf_fingerprints_erk.predict(X_untested_fingerprints)
df_y_pred_fingerprint_erk=pd.DataFrame(y_pred_fingerprint_erk,index=untested_molecules.index)

## Unison

In [25]:
intersect_pmk=untested_molecules[(df_y_pred_descriptor_pmk[0]==1)|(df_y_pred_fingerprint_pmk[0]==1)].index
len(intersect_pmk)

intersect_erk=untested_molecules[(df_y_pred_descriptor_erk[0]==1)|(df_y_pred_fingerprint_erk[0]==1)].index
len(intersect_erk)


62

Writing outcomes to CSV file

In [26]:

df_untested=pd.read_csv('untested_molecules.csv')
for i in range(len(df_untested['SMILES'])):
    molecule=df_untested.loc[i,'SMILES']
    if molecule in intersect_erk:
        df_untested.loc[i,kinase_ERK2]=1
        print(molecule)
    else:
        df_untested.loc[i,kinase_ERK2]=0

df_untested.set_index('SMILES', inplace=True)
df_untested.to_csv('predicted_molecules.csv')



df_untested=pd.read_csv('untested_molecules.csv')
for i in range(len(df_untested['SMILES'])):
    molecule=df_untested.loc[i,'SMILES']
    if molecule in intersect_pmk:
        df_untested.loc[i,kinase_PKM2]=1
        print(molecule)
    else:
        df_untested.loc[i,kinase_PKM2]=0

df_untested.set_index('SMILES', inplace=True)
df_untested.to_csv('predicted_molecules.csv')


O=P([O-])(CP(=O)(c1ccccc1)c1ccccc1)c1ccccc1
O=c1nc[nH]c2c1ncn2[C@H]1CC[C@@H](CO)O1
C=CC[C@@](O)(CC[NH+]1CCCCC1)c1ccccc1
c1cc(CNc2ccc3c(c2)OCO3)ccn1
O=P(Nc1nccs1)(c1ccccc1)c1ccccc1
C[NH+](C)CCOC(c1ccccc1)c1ccccc1
O/N=C1/CCc2nonc2/C1=N/O
O=C(N1CCCCCC1)N1CCN(C(=O)N2CCCCC2)CC1
O=c1c2ccccc2nc2n1CCS2
c1nc(N2CCCC2)c2[nH]cnc2n1
COc1ccc(CN2CC[NH2+]CC2)c(OC)c1OC
COc1cc(Br)cc(/C=N/c2ncn[nH]2)c1O
C=CCNc1ncnc2ccc(Br)cc12
COc1ccc(S(=O)(=O)c2ccc(OC)c(OC)c2)cc1OC
OCc1ccccc1C[NH2+]Cc1ccccc1
CN1C[N@H+](C)CSC1=S
C(=N/c1c(-c2ccccc2)nc2n1CCS2)\c1ccccc1
COc1cc(-c2ccccc2)nc(N)n1
C[C@@H]1C(=O)[C@@H](C)[C@@H](c2ccc3c(c2)OCO3)[NH2+][C@@H]1c1ccc2c(c1)OCO2
N#CC1=C2C=CC=CN2CC1=O
Oc1ccc(Nc2nc(N3CCCC3)c3ccccc3n2)cc1
COCCNc1nc([S-])nc2cc(OC)c(OC)cc12
O=P([O-])([O-])c1ccccc1OCCOCCOc1ccccc1P(=O)([O-])[O-]
COc1cc(C(=O)Nc2nc(-c3cccs3)cs2)cc(OC)c1OC
C[n+]1cnn2c1SCC1(COC1)C2
COc1ccc(-c2cc(N)n(S(=O)(=O)c3c(F)c(F)c(F)c(F)c3F)n2)cc1OC
c1ccc(CNc2ncnc3ccccc23)cc1
C1COC(NC(C2CC2)C2CC2)=[NH+]1
O[C@H](c1ccc2c(c1)OCO2)c1nccc2ccccc1